# ICC305 Avaliação de Desempenho - 2023/1

## Atividade 3

**Giovanna** Andrade Santos - 22052563

**Marcos** Avner Pimenta de Lima - 21852448

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import math
import time

from scipy.stats import chisquare
from scipy.stats import chi2_contingency

from ad_library import myfunctions
from collections import defaultdict

## Parte 1 - MM1

In [2]:
#nº de clientes
n1 = 10 ** 3
n2 = 10 ** 6
n3 = 10 ** 9
n4 = 10 ** 12

#taxa de entrada --> 9 clientes/s
tc = 9

#taxa de serviço --> 10 clientes/s
ts = 10

In [33]:

def mm1(n=1000, tc=9., ts=10., gerador=myfunctions.G5RandomGenerator()):
    lamb = 1. / tc
    mu = 1. / ts
    fila = list() # fila de espera
    # espera = np.zeros(n)
    # o vetor de tempos de espera irá armazenar valores em milisegundos (inteiros)
    # isso é para economizar memória RAM
    espera = np.zeros(n, dtype='uint16')
    cont = 0
    
    prox_chegada = myfunctions.va_exp(lamb, gerador) # computa o tempo da proxima chegada.
    prox_atend = prox_chegada + myfunctions.va_exp(mu, gerador) # computa o tempo da conclusao do proximo atendimento.
    
    # inicia a simulação da fila MM1.
    while n > 0:

        while prox_chegada < prox_atend:
            # simula a proxima chegada
            fila.append(prox_chegada)
            prox_chegada += myfunctions.va_exp(lamb, gerador)

        # Next event is a service completion.
        if fila:
            chegada = fila.pop(0)
            # para redução do consumo de RAM os tempos de espera são convertidos para milisegundos (inteiros)
            e = int((prox_atend - chegada) * 1000)
            espera[cont] = e if e > 0 else 0
            # espera[cont] = prox_atend - chegada
            cont += 1

        # atualiza a fila de espera.
        if fila:
            # se a fila não estiver vazia, o termino do proximo atendimento 
            # é incrementando com o ts
            prox_atend += myfunctions.va_exp(mu, gerador)
        else:
            # se a fila estiver vazia, o termino do próximo atendimento
            # é incrementando com o (tc + ts)
            prox_atend = prox_chegada + myfunctions.va_exp(mu, gerador)
        
        n -= 1
    return espera

In [34]:
#função para calcular intervalo de confiança
def ic(n, media, values: list, alpha=.05):
    std = np.std(values)
    H = st.t.ppf(q=1. - alpha / 2., df=n-1) * std
    li = media - H
    ls = media + H
    return (li, ls)

In [18]:
g5 = myfunctions.G5RandomGenerator()
g5.reset() # reinicia o gerador com a seed para replicação de resultados

for n in [n1, n2, n3]:    
    print(f'Iniciando simulação para n={n}')
    start = time.time()
    tot_espera = mm1(n, tc, ts, g5)
    end = time.time()
    print(f'Fim da simulação!')
    print(f'Tempo: {end-start} ms')
    
    print('Salvando tempos de espera')
    with open(f'mm1_n{n}', 'wb') as f:
        np.save(f, tot_espera)
        
    print('Calculando intervalo de confiança')
    soma = np.sum(tot_espera)
    media = myfunctions.mean(tot_espera)
    inter_conf = ic(n, media, tot_espera)
    
    print(f'N: {n}')
    print(f'Espera total: {soma} ms')
    print(f'Espera Média: {media} ms')
    print(f'Espera IC: {inter_conf} ms')
    print()

Iniciando simulação para n=1000
Fim da simulação!
Tempo: 0.055075883865356445ms
Salvando tempos de espera
Calculando intervalo de confiança
N: 1000
Espera total: 673838 ms
Espera Média: 673.838 ms
Espera IC: (-396.5929554419736, 1744.2689554419735) ms

Iniciando simulação para n=1000000
Fim da simulação!
Tempo: 0.9812638759613037ms
Salvando tempos de espera
Calculando intervalo de confiança
N: 1000000
Espera total: 1003611329 ms
Espera Média: 1003.611329 ms
Espera IC: (-961.434759208849, 2968.657417208849) ms

Iniciando simulação para n=1000000000
Fim da simulação!
Tempo: 1006.5161719322205ms
Salvando tempos de espera
Calculando intervalo de confiança
N: 1000000000
Espera total: 1000629351149 ms
Espera Média: 1000.629351149 ms
Espera IC: (-964.2031208617949, 2965.4618231597947) ms



In [23]:
valor_esperado = myfunctions.valor_esperado(tc, ts)
print(f'O valor esperado do tempo de espera para os valores de lambda e mi é {valor_esperado:.5} segundos')

O valor esperado do tempo de espera para os valores de lambda e mi é 0.9 segundos


O valor médio esperado é de aproximadamente `0.9 segundos`.

Para `n=1000` o tempo médio $\overline{X}(n)$ foi de aproximadamente `0.673 segundos` bem longe do valor esperado, embora o intervalo de confiança `[-0.396, 1.733]`  contenha o valor esperado.

Para `n=1000000` o tempo médio $\overline{X}(n)$ foi de aproximadamente `1.003 segundos`, mais próximo do valor esperado, e o intervalo de confiança `[-0.961, 2.968]` também contem o valor esperado.

Para `n=1000000000` o tempo médio $\overline{X}(n)$ foi de aproximadamente `1.000 segundos`, um pouco mais próximo do valor esperado, e novamente intervalo de confiança `[-0.961, 2.968]` contem o valor esperado.

In [26]:
e = np.zeros(10)
e[1] = 3
e[5] = -1
e[5] = 2
e

array([0., 3., 0., 0., 0., 2., 0., 0., 0., 0.])

In [27]:
e = np.resize(e, (50, ))
e

array([0., 3., 0., 0., 0., 2., 0., 0., 0., 0., 0., 3., 0., 0., 0., 2., 0.,
       0., 0., 0., 0., 3., 0., 0., 0., 2., 0., 0., 0., 0., 0., 3., 0., 0.,
       0., 2., 0., 0., 0., 0., 0., 3., 0., 0., 0., 2., 0., 0., 0., 0.])

## Parte 2 - Chow e Robbins

In [47]:

def mm1_v2(d, batch=10, tc=9., ts=10., gerador=myfunctions.G5RandomGenerator()):
    lamb = 1. / tc
    mu = 1. / ts
    fila = list() # fila de espera
    H = 0
    n = 10
    
    # o vetor de tempos de espera irá armazenar valores em milisegundos (inteiros)
    # isso é para economizar memória RAM
    espera = np.zeros(n)
    cont = 0   
    
    # inicia a simulação da fila MM1 (Chow e Robbins)
    while H <= d:

        prox_chegada = myfunctions.va_exp(lamb, gerador) # computa o tempo da proxima chegada.
        prox_atend = prox_chegada + myfunctions.va_exp(mu, gerador) # computa o tempo da conclusao do proximo atendimento.
        x = cont
        
        while x < n:
            while prox_chegada < prox_atend:
                # simula a proxima chegada
                fila.append(prox_chegada)
                prox_chegada += myfunctions.va_exp(lamb, gerador)

            # Next event is a service completion.
            if fila:
                chegada = fila.pop(0)
                e = prox_atend - chegada
                # esperas negativas, significam que a chega ocorre apos o termino do atendimento
                # logo não houve espera, por isso adotamos como zero
                espera[cont] = e if e > 0 else 0.
                # espera[cont] = prox_atend - chegada
                cont += 1

            # atualiza a fila de espera.
            if fila:
                # se a fila não estiver vazia, o termino do proximo atendimento 
                # é incrementando com o ts
                prox_atend += myfunctions.va_exp(mu, gerador)
            else:
                # se a fila estiver vazia, o termino do próximo atendimento
                # é incrementando com o (tc + ts)
                prox_atend = prox_chegada + myfunctions.va_exp(mu, gerador)
            x += 1
        
        std = np.std(espera)
        H = st.t.ppf(q=1. - .05 / 2., df=n-1) * std
        print(f'H={H}, n={n}')
        n += batch
        espera = np.resize(espera, (n, ))
        
    return (n, espera)

### d = 0.1

In [52]:
g5 = myfunctions.G5RandomGenerator()
g5.reset() # reinicia o gerador com a seed para replicação de resultados

n = mm1_v2(.1, 1, tc, ts, g5)
n

H=0.23086778937644267, n=10


(11,
 array([0.14186292, 0.13313644, 0.13598471, 0.12117453, 0.08755856,
        0.07025651, 0.12980219, 0.02883574, 0.40620018, 0.25867514,
        0.14186292]))

### d = 0.5

In [59]:
g5 = myfunctions.G5RandomGenerator()
g5.reset() # reinicia o gerador com a seed para replicação de resultados

n = mm1_v2(.5, 7, tc, ts, g5)
n

H=0.23086778937644267, n=10
H=0.3011094551839147, n=17
H=0.28855613282764486, n=24
H=0.29819283107558786, n=31
H=0.30455934052523886, n=38
H=0.3267244033476495, n=45
H=0.3295927748943015, n=52
H=0.38024674663601704, n=59
H=0.3739906206584639, n=66
H=0.38116995330094716, n=73
H=0.3663896751241286, n=80
H=0.35989216998537715, n=87
H=0.3666854475682198, n=94
H=0.37072702323926643, n=101
H=0.38218472093305955, n=108
H=0.42992655286660963, n=115
H=0.43204565667668926, n=122
H=0.436471643850262, n=129
H=0.44020870085474306, n=136
H=0.436148591456043, n=143
H=0.4384290746025827, n=150
H=0.4340625566737429, n=157
H=0.4867423056975035, n=164
H=0.4876878472974787, n=171
H=0.48143247505058545, n=178
H=0.479209928776354, n=185
H=0.47907707714700715, n=192
H=0.49173840014608794, n=199
H=0.4988220683465792, n=206
H=0.4966918316866844, n=213
H=0.49173604847424696, n=220
H=0.4948565917028831, n=227
H=0.491539592081621, n=234
H=0.4897289643187061, n=241
H=0.4841065228615082, n=248
H=0.48472663786173126

(297,
 array([0.14186292, 0.13313644, 0.13598471, 0.12117453, 0.08755856,
        0.07025651, 0.12980219, 0.02883574, 0.40620018, 0.25867514,
        0.        , 0.15828147, 0.48642715, 0.45333034, 0.34548972,
        0.27488245, 0.17932039, 0.        , 0.        , 0.13533261,
        0.20438523, 0.3433716 , 0.30965373, 0.28774954, 0.27005855,
        0.42568009, 0.42907233, 0.38142075, 0.35220449, 0.35656006,
        0.04859723, 0.02066915, 0.10690323, 0.24435409, 0.4187043 ,
        0.48968773, 0.28203662, 0.04964233, 0.18936989, 0.08911133,
        0.08667747, 0.57894653, 0.5538756 , 0.46389127, 0.3011708 ,
        0.        , 0.        , 0.        , 0.        , 0.19110771,
        0.24388147, 0.15180949, 0.        , 0.47961059, 0.43769522,
        0.4099785 , 0.68227264, 0.78394987, 0.42741226, 0.        ,
        0.        , 0.11635849, 0.06073201, 0.16160339, 0.31486662,
        0.32904416, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        

### d = 1.0

In [68]:
g5 = myfunctions.G5RandomGenerator()
g5.reset() # reinicia o gerador com a seed para replicação de resultados

n = mm1_v2(1., 18, tc, ts, g5)
n

H=0.23086778937644267, n=10
H=0.2781649694580492, n=28
H=0.3192451536219535, n=46
H=0.45452527750406513, n=64
H=0.5308695462957245, n=82
H=0.6258592867544454, n=100
H=0.6851490297343493, n=118
H=0.6625400517036096, n=136
H=0.9544261845522064, n=154
H=0.9374787351143783, n=172
H=1.013423243364637, n=190


(208,
 array([0.14186292, 0.13313644, 0.13598471, 0.12117453, 0.08755856,
        0.07025651, 0.12980219, 0.02883574, 0.40620018, 0.25867514,
        0.        , 0.15828147, 0.48642715, 0.45333034, 0.34548972,
        0.27488245, 0.17932039, 0.2012262 , 0.14479979, 0.06803847,
        0.0279596 , 0.20438523, 0.3433716 , 0.30965373, 0.28774954,
        0.03857428, 0.27005855, 0.42568009, 0.        , 0.        ,
        0.        , 0.        , 0.18343556, 0.17752661, 0.14701002,
        0.23759923, 0.41194944, 0.48293286, 0.27528176, 0.04964233,
        0.03224587, 0.18936989, 0.08911133, 0.08667747, 0.57894653,
        0.5538756 , 0.        , 0.        , 0.        , 0.        ,
        0.43107722, 0.49363421, 0.58008947, 0.40693842, 0.49594244,
        0.56415087, 0.82505501, 0.577939  , 0.49799938, 0.71927017,
        0.69286072, 0.73186528, 0.7493187 , 0.45880827, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        

### d = 2.0

In [86]:
g5 = myfunctions.G5RandomGenerator()
g5.reset() # reinicia o gerador com a seed para replicação de resultados

n = mm1_v2(2., 50, tc, ts, g5)
n

H=0.23086778937644267, n=10
H=0.29920699257705013, n=60
H=0.42382495737251163, n=110
H=1.4888038093728433, n=160
H=2.5414634872637962, n=210


(260,
 array([1.41862915e-01, 1.33136440e-01, 1.35984714e-01, 1.21174533e-01,
        8.75585639e-02, 7.02565102e-02, 1.29802187e-01, 2.88357422e-02,
        4.06200184e-01, 2.58675140e-01, 0.00000000e+00, 1.58281475e-01,
        4.86427153e-01, 4.53330341e-01, 3.45489716e-01, 2.74882448e-01,
        1.79320391e-01, 2.01226196e-01, 1.44799790e-01, 6.80384701e-02,
        2.79596022e-02, 2.04385228e-01, 3.43371604e-01, 3.09653727e-01,
        2.87749541e-01, 3.85742785e-02, 2.70058549e-01, 4.25680086e-01,
        4.29072335e-01, 3.81420748e-01, 3.52204494e-01, 3.56560063e-01,
        4.85972286e-02, 3.42954505e-02, 1.05839931e-01, 1.47601950e-01,
        2.38191160e-01, 4.12541369e-01, 4.83524797e-01, 2.75873691e-01,
        4.96423255e-02, 3.22458657e-02, 1.89369894e-01, 8.91113319e-02,
        8.66774666e-02, 5.78946528e-01, 5.53875599e-01, 4.63891265e-01,
        3.01170797e-01, 3.16524407e-01, 4.00751649e-01, 3.00838496e-01,
        2.85725215e-01, 2.59253016e-01, 4.55146479e-01, 2.

## Parte 3

In [94]:

def mm1_v3(gamma, batch=10, tc=9., ts=10., gerador=myfunctions.G5RandomGenerator()):
    lamb = 1. / tc
    mu = 1. / ts
    fila = list() # fila de espera
    H = 0
    n = 10
    
    # o vetor de tempos de espera irá armazenar valores em milisegundos (inteiros)
    # isso é para economizar memória RAM
    espera = np.zeros(n)
    cont = 0
    xn = 0.0000000000000000000001
    
    # inicia a simulação da fila MM1 (Chow e Robbins)
    while H/xn <= gamma:

        prox_chegada = myfunctions.va_exp(lamb, gerador) # computa o tempo da proxima chegada.
        prox_atend = prox_chegada + myfunctions.va_exp(mu, gerador) # computa o tempo da conclusao do proximo atendimento.
        x = cont
        
        while x < n:
            while prox_chegada < prox_atend:
                # simula a proxima chegada
                fila.append(prox_chegada)
                prox_chegada += myfunctions.va_exp(lamb, gerador)

            # Next event is a service completion.
            if fila:
                chegada = fila.pop(0)
                e = prox_atend - chegada
                # esperas negativas, significam que a chega ocorre apos o termino do atendimento
                # logo não houve espera, por isso adotamos como zero
                espera[cont] = e if e > 0 else 0.
                # espera[cont] = prox_atend - chegada
                cont += 1

            # atualiza a fila de espera.
            if fila:
                # se a fila não estiver vazia, o termino do proximo atendimento 
                # é incrementando com o ts
                prox_atend += myfunctions.va_exp(mu, gerador)
            else:
                # se a fila estiver vazia, o termino do próximo atendimento
                # é incrementando com o (tc + ts)
                prox_atend = prox_chegada + myfunctions.va_exp(mu, gerador)
            x += 1
        
        xn = myfunctions.mean(espera)
        std = np.std(espera)
        H = st.t.ppf(q=1. - .05 / 2., df=n-1) * std
        print(f'H={H}, X(n)={xn}, n={n}')
        n += batch
        espera = np.resize(espera, (n, ))
        
    return (H, xn, n, espera)

In [98]:
g5 = myfunctions.G5RandomGenerator()
g5.reset() # reinicia o gerador com a seed para replicação de resultados

H, Xn, n, espera = mm1_v3(.05, 1, tc, ts, g5)

print(f'H = {H}')
print(f'Xn = {Xn}')
print(f'n = {n}')
print()

H=0.23086778937644267, X(n)=0.1513486929723388, n=10
H = 0.23086778937644267
Xn = 0.1513486929723388
n = 11

